# General imports and functions

In [87]:
from pathlib import Path
import numpy as np
from collections import defaultdict
import re

DATA_ROOT = Path("data")

# Day 1

In [44]:
file = DATA_ROOT / 'day_1' / 'full.txt'
contents = np.genfromtxt(file, dtype=np.int64)

## Part 1

In [31]:
contents[:, 0].sort()
contents[:, 1].sort()
np.abs(np.diff(contents, axis=1)).sum().item()

1941353

## Part 2

In [45]:
numbers, counts = np.unique(contents[:, 1], return_counts=True)
occurences = dict(zip(numbers, counts))
s = np.sum(contents[:, 0] * np.vectorize(lambda x: occurences.get(x, 0))(contents[:, 0]))
s.item()

22539317

# Day 2

In [80]:
file = DATA_ROOT / 'day_2' / 'full.txt'
contents = [[int(x) for x in l.split()] for l in open(file, 'r').readlines()]

## Part 1

In [66]:
s = 0
for report in contents:
    report = np.array(report)
    ascending = np.all(report[:-1] <= report[1:])
    descending = np.all(report[:-1] >= report[1:])
    differences = np.abs(np.diff(report))
    level_changes = (differences.min() >= 1) & (differences.max() <= 3)
    s += (ascending | descending) & level_changes
s.item()

332

## Part 2

In [81]:
def check_report(report):
    ascending = np.all(report[:-1] <= report[1:])
    descending = np.all(report[:-1] >= report[1:])
    differences = np.abs(np.diff(report))
    level_changes = (differences.min() >= 1) & (differences.max() <= 3)
    return (ascending | descending) & level_changes

s = 0
for report in contents:
    report = np.array(report)
    if check_report(report):
        s += 1
    else:
        for i in range(len(report)):
            if check_report(np.concatenate((report[:i], report[i + 1:]))):
                s += 1
                break
s

398

# Day 3

In [101]:
file = DATA_ROOT / 'day_3' / 'test.txt'
contents = ''.join(open(file, 'r').readlines())

## Part 1

In [99]:
sum((int(s.split(',')[0][4:]) * int(s.split(',')[1][:-1]) for s in re.findall(r'mul\(\d{1,3},\d{1,3}\)', contents)))

161

## Part 2

In [ ]:
matches = re.findall(r'mul\(\d{1,3},\d{1,3}\)|do\(\)|don\'t\(\)', contents)
enabled = True
s = 0
for match in matches:
    if match == 'do()':
        enabled = True
    elif match == 'don\'t()':
        enabled = False
    elif enabled:
        s += int(s.split(',')[0][4:]) * int(s.split(',')[1][:-1])
s

['mul(2,4)', "don't()", 'mul(5,5)', 'mul(11,8)', 'do()', 'mul(8,5)']